<a href="https://colab.research.google.com/github/khadijabensaad/ExpressMongoose/blob/main/un.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [1]:
# -*- coding: utf-8 -*-
"""
Analyse des modalités NIfTI (T1, T1CE, T2, FLAIR, seg)
Objectif : déterminer quelles modalités sont les plus informatives pour la classification
(ici: séparation voxel tumeur vs fond). Génère des courbes prouvant l'analyse.

Usage:
 - Remplacer les chemins dans `paths` par vos fichiers .nii / .nii.gz
 - Lancer dans un Jupyter notebook ou en script
"""

import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tqdm import tqdm
from scipy import stats

# -----------------------
# CONFIG : chemins vers vos fichiers NIfTI
# -----------------------
paths = {
    "T1":  "data/T1.nii.gz",
    "T1CE":"data/T1CE.nii.gz",
    "T2":  "data/T2.nii.gz",
    "FLAIR":"data/FLAIR.nii.gz",
    "SEG": "data/seg.nii.gz",   # segmentation binaire (0=fond, >0=tumeur)
}

# -----------------------
# FONCTIONS UTILITAIRES
# -----------------------
def load_nifti(path):
    img = nib.load(path)
    data = img.get_fdata(dtype=np.float32)
    return data

def normalize_image(img, mask=None):
    """Z-score normalization using either whole image or masked voxels if mask provided."""
    if mask is not None:
        vals = img[mask > 0]
    else:
        vals = img.flatten()
    mean = np.mean(vals)
    std = np.std(vals) if np.std(vals) > 0 else 1.0
    imgn = (img - mean) / std
    return imgn

def compute_basic_stats(img, seg_mask):
    """Retourne mean_tumor, mean_background, ratio, std_tumor, snr approximation"""
    tumor = img[seg_mask > 0]
    bg = img[seg_mask == 0]
    mean_t = np.mean(tumor) if tumor.size>0 else 0.0
    mean_b = np.mean(bg) if bg.size>0 else 0.0
    std_t = np.std(tumor) if tumor.size>0 else 0.0
    ratio = (mean_t / (mean_b + 1e-9)) if mean_b != 0 else np.nan
    snr = mean_t / (np.std(bg) + 1e-9) if bg.size>0 else np.nan
    return {"mean_tumor": mean_t, "mean_bg": mean_b, "ratio_t_b": ratio, "std_tumor": std_t, "snr": snr}

def compute_mutual_info(img, seg_mask, n_bins=64, sample_size=200000):
    """Approximate mutual information between intensity and seg label (discrete)."""
    flat_img = img.flatten()
    flat_seg = (seg_mask.flatten() > 0).astype(int)
    # sample to speed up
    idx = np.arange(flat_img.size)
    if sample_size < flat_img.size:
        idx = np.random.choice(idx, size=sample_size, replace=False)
    a = flat_img[idx]
    b = flat_seg[idx]
    # discretize a into bins
    a_disc = np.digitize(a, bins=np.linspace(np.min(a), np.max(a), n_bins))
    mi = mutual_info_score(a_disc, b)
    return mi

# -----------------------
# LOAD
# -----------------------
print("Chargement des images...")
for k,p in paths.items():
    if not os.path.exists(p):
        raise FileNotFoundError(f"Fichier non trouvé : {p} (mettez le bon chemin)")
print("Tous les fichiers existent. Lecture en cours...")

img_T1   = load_nifti(paths["T1"])
img_T1CE = load_nifti(paths["T1CE"])
img_T2   = load_nifti(paths["T2"])
img_FLAIR= load_nifti(paths["FLAIR"])
img_seg  = load_nifti(paths["SEG"])

# Binariser la segmentation : 0 = fond, 1 = tumeur
seg_mask = (img_seg > 0.5).astype(np.uint8)

# -----------------------
# NORMALISATION (Z-score par modalité)
# -----------------------
print("Normalisation (Z-score) par modalité...")
img_T1_n    = normalize_image(img_T1)
img_T1CE_n  = normalize_image(img_T1CE)
img_T2_n    = normalize_image(img_T2)
img_FLAIR_n = normalize_image(img_FLAIR)

modalities = {
    "T1": img_T1_n,
    "T1CE": img_T1CE_n,
    "T2": img_T2_n,
    "FLAIR": img_FLAIR_n
}

# -----------------------
# STATISTIQUES BASIQUES & MI
# -----------------------
print("Calcul des statistiques basiques et mutual information...")
stats_list = []
mi_list = {}
for name, img in modalities.items():
    st = compute_basic_stats(img, seg_mask)
    mi = compute_mutual_info(img, seg_mask, n_bins=64, sample_size=200000)
    st["modalite"] = name
    st["mutual_info"] = mi
    stats_list.append(st)
    mi_list[name] = mi

df_stats = pd.DataFrame(stats_list).set_index("modalite")
print("\nStats par modalité :\n", df_stats.round(4))

# -----------------------
# EVALUATION par MODALITÉ : classification voxel tumeur vs fond (logistic regression)
# -----------------------
# Nous allons former un modèle simple par modalité : feature = intensity normalisée
# puis mesurer l'AUC pour la capacité à séparer tumeur / fond.
print("\nEvaluation AUC par modalité (logistic regression sur voxels, échantillonnage équilibré)...")

def prepare_voxel_dataset(img, seg_mask, max_samples=200000):
    flat_img = img.flatten()
    flat_seg = (seg_mask.flatten() > 0).astype(int)
    idx_tumor = np.where(flat_seg==1)[0]
    idx_bg = np.where(flat_seg==0)[0]
    n_pos = idx_tumor.size
    n_neg = idx_bg.size
    # équilibrer via sous-échantillonnage/oversampling si besoin
    n_sample = min(max_samples//2, n_pos)  # max pos
    if n_pos==0:
        raise ValueError("Aucune voxels de tumeur dans la segmentation.")
    pos_idx = np.random.choice(idx_tumor, size=n_sample, replace=False)
    neg_idx = np.random.choice(idx_bg, size=n_sample, replace=False)
    idx = np.concatenate([pos_idx, neg_idx])
    X = flat_img[idx].reshape(-1,1)
    y = flat_seg[idx]
    return X, y

auc_results = {}
roc_curves = {}

for name, img in modalities.items():
    X, y = prepare_voxel_dataset(img, seg_mask, max_samples=200000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    clf = LogisticRegression(max_iter=200, solver='lbfgs')
    clf.fit(X_train, y_train)
    probs = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, probs)
    fpr, tpr, _ = roc_curve(y_test, probs)
    auc_results[name] = auc
    roc_curves[name] = (fpr, tpr)
    print(f"AUC {name}: {auc:.4f}")

# -----------------------
# MODELE COMBINÉ (toutes modalités) + permutation importance approximée
# -----------------------
print("\nEvaluation modèle combiné (modalités concaténées) — logistic regression multivariée...")
# préparer X avec les 4 intensities
flat_shape = img_T1_n.shape
flat_size = flat_shape[0]*flat_shape[1]*flat_shape[2]
# pour la combinaison, échantillonnons voxels (équilibrés)
def prepare_combined_dataset(modals_dict, seg_mask, n_each=50000):
    flat_seg = (seg_mask.flatten() > 0).astype(int)
    idx_tumor = np.where(flat_seg==1)[0]
    idx_bg = np.where(flat_seg==0)[0]
    n_pos = min(len(idx_tumor), n_each)
    n_neg = min(len(idx_bg), n_each)
    pos_idx = np.random.choice(idx_tumor, size=n_pos, replace=False)
    neg_idx = np.random.choice(idx_bg, size=n_neg, replace=False)
    idx = np.concatenate([pos_idx, neg_idx])
    X = np.vstack([modals_dict[m].flatten()[idx] for m in modals_dict.keys()]).T
    y = flat_seg[idx]
    return X, y

Xc, yc = prepare_combined_dataset(modalities, seg_mask, n_each=60000)
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, stratify=yc, random_state=42)
clf_comb = LogisticRegression(max_iter=400, solver='lbfgs')
clf_comb.fit(Xc_train, yc_train)
probs_comb = clf_comb.predict_proba(Xc_test)[:,1]
auc_comb = roc_auc_score(yc_test, probs_comb)
print(f"AUC modèle combiné : {auc_comb:.4f}")

# Permutation importance approximée : mesurer chute d'AUC quand on permute une colonne
print("Calcul permutation importance (approx.)...")
perm_importance = {}
rng = np.random.RandomState(42)
baseline_auc = auc_comb
for i, m in enumerate(list(modalities.keys())):
    Xp = Xc_test.copy()
    Xp[:, i] = rng.permutation(Xp[:, i])  # permute colonne i
    auc_p = roc_auc_score(yc_test, clf_comb.predict_proba(Xp)[:,1])
    perm_importance[m] = baseline_auc - auc_p  # perte d'AUC si modalité permutée

# -----------------------
# PLOTS : AUC par modalité, MI, ROC curves, Permutation importance
# -----------------------
plt.style.use('default')
fig, axes = plt.subplots(2,2, figsize=(12,10))

# AUC barplot
ax = axes[0,0]
names = list(auc_results.keys())
aucs = [auc_results[n] for n in names]
ax.bar(names, aucs)
ax.set_ylim(0.0,1.0)
ax.set_title("AUC par modalité (séparation voxels tumeur vs fond)")
ax.set_ylabel("AUC")
for i, v in enumerate(aucs):
    ax.text(i, v+0.02, f"{v:.3f}", ha='center')

# Mutual information barplot
ax = axes[0,1]
names_mi = list(mi_list.keys())
mis = [mi_list[n] for n in names_mi]
ax.bar(names_mi, mis)
ax.set_title("Information mutuelle (intensité ⟷ segmentation)")
ax.set_ylabel("MI (bits approx.)")
for i, v in enumerate(mis):
    ax.text(i, v+0.01, f"{v:.3f}", ha='center')

# ROC curves (top 3 modalities)
ax = axes[1,0]
top_modalities = sorted(auc_results.items(), key=lambda x: x[1], reverse=True)[:3]
for name, _ in top_modalities:
    fpr, tpr = roc_curves[name]
    ax.plot(fpr, tpr, label=f"{name} (AUC={auc_results[name]:.3f})")
ax.plot([0,1],[0,1],'k--', linewidth=0.6)
ax.set_title("ROC curves (top 3 modalités)")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.legend()

# Permutation importance
ax = axes[1,1]
mods = list(perm_importance.keys())
vals = [perm_importance[m] for m in mods]
ax.bar(mods, vals)
ax.set_title("Permutation importance (chute d'AUC si permuté)")
ax.set_ylabel("Perte d'AUC")
for i, v in enumerate(vals):
    ax.text(i, v+0.001, f"{v:.4f}", ha='center')

plt.tight_layout()
plt.show()

# -----------------------
# RÉSULTATS RÉCAPITULATIFS
# -----------------------
print("\n--- RÉSUMÉ ---")
print("AUC par modalité :")
for k,a in auc_results.items():
    print(f"  {k}: AUC = {a:.4f}, MI = {mi_list[k]:.4f}, mean_tumor={df_stats.loc[k,'mean_tumor']:.3f}, mean_bg={df_stats.loc[k,'mean_bg']:.3f}")

print(f"\nModèle combiné AUC = {auc_comb:.4f}")
print("Permutation importance (perte d'AUC si permutée) :")
for k,v in perm_importance.items():
    print(f"  {k}: {v:.4f}")

# -----------------------
# CONSEILS D'INTERPRÉTATION (affichés ici pour référence)
# -----------------------
# - Une modalité avec AUC élevé et MI élevé sépare bien tumeur/fond — utile pour classification & génération.
# - Si la modalité T1CE (par exemple) a la meilleure AUC → elle est cruciale pour la classification
# - Le modèle combiné doit dépasser les modalités individuelles ; la permutation importance
#   indique quelles modalités contribuent le plus.
#
# Vous pouvez ensuite :
# - utiliser les modalités top-N pour entraîner votre CNN3D + EfficientNet (ex : Empiler ces canaux en entrée)
# - ou utiliser Med-DDPM conditionné sur ces canaux pour la simulation post-traitement.
#
# Fin du script.


Chargement des images...


FileNotFoundError: Fichier non trouvé : data/T1.nii.gz (mettez le bon chemin)